In [ ]:
  %matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
import imblearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score

In [ ]:
#Ignorando avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Formatação
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold= 15)
np.set_printoptions(precision=3)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
df = pd.read_csv('../datasets/C_Dados_V5.csv')
df

,ts,date,time,fridge_temperature,temp_condition,door_state,sphone_signal,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity,label,type
0,1556245180,25-Apr-19,19:19:40,9.000000,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,1,ddos
1,1556245180,25-Apr-19,19:19:40,9.250000,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,1,ddos
2,1556245185,25-Apr-19,19:19:45,12.650000,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,1,ddos
3,1556245185,25-Apr-19,19:19:45,4.650000,low,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,1,ddos
4,1556245195,25-Apr-19,19:19:55,12.650000,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,1,ddos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401114,1554166618,1-Apr-19,17:56:58,7.131876,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,32.799434,2.204924,37.024913,0,normal
401115,1554166618,1-Apr-19,17:56:58,7.131876,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,29.453781,-2.030547,90.297894,0,normal
401116,1554166619,1-Apr-19,17:56:59,7.131876,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,47.185992,0.872942,37.687701,0,normal
401117,1554166623,1-Apr-19,17:57:03,7.131876,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,43.097037,3.168207,93.647950,0,normal


In [ ]:
# Criando outras features usando a hora (hour-minute-second)
df[['hour', 'minute', 'second']] = df['time'].str.split(':', expand=True)
df['hour'] = df['hour'].astype(int)
df['minute'] = df['minute'].astype(int)
df['second'] = df['second'].astype(int)

# Criando outras features usando a data (day-month-year)
df.date = pd.to_datetime(df.date)
df['day'] = df.date.dt.day

# Removendo features
df.drop(['ts', 'date', 'time', 'label'], axis=1, inplace=True)
df.head()

,fridge_temperature,temp_condition,door_state,sphone_signal,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity,type,hour,minute,second,day
0,9.00,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,40,25
1,9.25,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,40,25
2,12.65,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,45,25
3,4.65,low,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,45,25
4,12.65,high,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,55,25


In [ ]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Crie uma instância do OrdinalEncoder
encoder = OrdinalEncoder()

# Ajuste e transforme a coluna "temp_condition"
df['temp_condition_encoded'] = encoder.fit_transform(df[['temp_condition']])

# Remova a coluna original "temp_condition" se necessário
df = df.drop('temp_condition', axis=1)
df

,fridge_temperature,door_state,sphone_signal,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity,type,hour,minute,second,day,temp_condition_encoded
0,9.000000,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,40,25,0.0
1,9.250000,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,40,25,0.0
2,12.650000,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,45,25,0.0
3,4.650000,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,45,25,1.0
4,12.650000,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,35.378616,0.892187,48.729767,ddos,19,19,55,25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401114,7.131876,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,32.799434,2.204924,37.024913,normal,17,56,58,1,0.0
401115,7.131876,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,29.453781,-2.030547,90.297894,normal,17,56,58,1,0.0
401116,7.131876,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,47.185992,0.872942,37.687701,normal,17,56,59,1,0.0
401117,7.131876,closed,0,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,0,28.76238,1,43.097037,3.168207,93.647950,normal,17,57,3,1,0.0


In [ ]:
df['door_state_encoded'] = encoder.fit_transform(df[['door_state']])
df['sphone_signal_encoded'] = encoder.fit_transform(df[['sphone_signal']])
df['light_status_encoded'] = encoder.fit_transform(df[['light_status']])
df['thermostat_status_encoded'] = encoder.fit_transform(df[['thermostat_status']])

df = df.drop('door_state', axis=1)
df = df.drop('sphone_signal', axis=1)
df = df.drop('light_status', axis=1)
df = df.drop('thermostat_status', axis=1)

df.head(3)

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,current_temperature,temperature,pressure,humidity,type,hour,minute,second,day,temp_condition_encoded,door_state_encoded,sphone_signal_encoded,light_status_encoded,thermostat_status_encoded
0,9.00,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,ddos,19,19,40,25,0.0,0.0,0.0,0.0,1.0
1,9.25,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,ddos,19,19,40,25,0.0,0.0,0.0,0.0,1.0
2,12.65,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,ddos,19,19,45,25,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Separando as Features do Label
y_data = df.type
X_data = df.drop(['type'], axis=1)

print("X:", X_data.shape)
X_data.head(3)

X: (401119, 21)


,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,current_temperature,temperature,pressure,humidity,hour,minute,second,day,temp_condition_encoded,door_state_encoded,sphone_signal_encoded,light_status_encoded,thermostat_status_encoded
0,9.00,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
1,9.25,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
2,12.65,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,45,25,0.0,0.0,0.0,0.0,1.0


In [ ]:
X_data.head()

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,current_temperature,temperature,pressure,humidity,hour,minute,second,day,temp_condition_encoded,door_state_encoded,sphone_signal_encoded,light_status_encoded,thermostat_status_encoded
0,9.00,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
1,9.25,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
2,12.65,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,45,25,0.0,0.0,0.0,0.0,1.0
3,4.65,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,45,25,1.0,0.0,0.0,0.0,1.0
4,12.65,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,55,25,0.0,0.0,0.0,0.0,1.0


In [ ]:
y_data = LabelEncoder().fit_transform(y_data)

In [ ]:
# determine categorical and numerical features
numerical_ix = X_data.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X_data.select_dtypes(include=['object', 'bool']).columns

In [ ]:
numerical_ix

Index(['fridge_temperature', 'latitude', 'longitude',
       'FC1_Read_Input_Register', 'FC2_Read_Discrete_Value',
       'FC3_Read_Holding_Register', 'FC4_Read_Coil', 'motion_status',
       'current_temperature', 'temperature', 'pressure', 'humidity', 'day',
       'temp_condition_encoded', 'door_state_encoded', 'sphone_signal_encoded',
       'light_status_encoded', 'thermostat_status_encoded'],
      dtype='object')

In [ ]:
categorical_ix

Index([], dtype='object')

In [ ]:
X_data

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,current_temperature,temperature,pressure,humidity,hour,minute,second,day,temp_condition_encoded,door_state_encoded,sphone_signal_encoded,light_status_encoded,thermostat_status_encoded
0,9.000000,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
1,9.250000,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,40,25,0.0,0.0,0.0,0.0,1.0
2,12.650000,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,45,25,0.0,0.0,0.0,0.0,1.0
3,4.650000,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,45,25,1.0,0.0,0.0,0.0,1.0
4,12.650000,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,35.378616,0.892187,48.729767,19,19,55,25,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401114,7.131876,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,32.799434,2.204924,37.024913,17,56,58,1,0.0,0.0,0.0,0.0,1.0
401115,7.131876,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,29.453781,-2.030547,90.297894,17,56,58,1,0.0,0.0,0.0,0.0,1.0
401116,7.131876,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,47.185992,0.872942,37.687701,17,56,59,1,0.0,0.0,0.0,0.0,1.0
401117,7.131876,46.703771,56.840204,32757.773627,32777.699879,32503.208743,32828.110594,0,28.76238,43.097037,3.168207,93.647950,17,57,3,1,0.0,0.0,0.0,0.0,1.0


In [ ]:
y_data

array([1, 1, 1, ..., 3, 3, 3])

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print (classification_report(y_true, y_pred) )
    return accuracy_score(y_true, y_pred)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn import tree

est_RL  = LogisticRegression(C = 0.1, 
                             class_weight = 'balanced', 
                             max_iter = 100, 
                             multi_class = 'multinomial', 
                             penalty= 'l2', 
                             solver = 'newton-cg')

est_KNN = KNeighborsClassifier(metric = 'euclidean', 
                               n_neighbors = 7)


est_GBM = GradientBoostingClassifier(learning_rate = 0.1, 
                                     max_depth = 3, 
                                     max_features = 'sqrt', 
                                     min_samples_leaf = 2, 
                                     min_samples_split = 4, 
                                     n_estimators = 100, 
                                     subsample = 1.0)

est_NB  = BernoulliNB(alpha = 0.1)

est_LDA = LinearDiscriminantAnalysis(solver = 'svd')

est_DTC = DecisionTreeClassifier(class_weight =  None, 
                                 criterion = 'entropy', 
                                 max_depth = 2, 
                                 max_features = 'sqrt', 
                                 min_samples_leaf = 1, 
                                 min_samples_split = 2)

est_RF  = RandomForestClassifier(bootstrap = True, 
                                 max_depth = 7, 
                                 max_features = 'sqrt', 
                                 min_samples_leaf = 1,
                                 min_samples_split = 2, 
                                 n_estimators = 100)

est_Ensemble = VotingClassifier(estimators=[('RL', est_RL), ('KNN', est_KNN), ('GBM', est_GBM), ('NB', est_NB), ('LDA', est_LDA), ('DTC', est_DTC), ('RF', est_RF)],
                        voting='soft')

In [ ]:
from sklearn.pipeline import make_pipeline

clf = make_pipeline(est_Ensemble)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score

# Realize a validação cruzada e obtenha as previsões do modelo
y_pred = cross_val_predict(clf, X_data, y_data, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Calcule e exiba o classification report para cada estimador
for name, estimator in est_Ensemble.estimators:
    estimator.fit(X_data, y_data)  # Treine o estimador
    y_pred_estimator = estimator.predict(X_data)  # Faça previsões
    report = classification_report(y_data, y_pred_estimator)
    print(f"Estimador: {name}")
    print(report)
    print("---")

# Encontre o estimador com o melhor desempenho
best_estimator_name = None
best_estimator_score = 0.0

for name, estimator in est_Ensemble.estimators:
    estimator.fit(X_data, y_data)  # Treine o estimador
    y_pred_estimator = estimator.predict(X_data)  # Faça previsões
    accuracy = accuracy_score(y_data, y_pred_estimator)
    if accuracy > best_estimator_score:
        best_estimator_score = accuracy
        best_estimator_name = name

# Exiba o melhor estimador
print("Melhor estimador: ", best_estimator_name)
print("Pontuação de acurácia: ", best_estimator_score)


Estimador: RL
              precision    recall  f1-score   support

           0       0.08      0.38      0.14     35000
           1       0.46      0.41      0.43     25000
           2       0.73      0.94      0.83     35000
           3       0.90      0.36      0.52    245000
           4       0.56      0.49      0.52     35000
           5       0.80      0.93      0.86     16030
           6       0.14      0.67      0.23      3973
           7       0.70      0.94      0.81      6116

    accuracy                           0.46    401119
   macro avg       0.55      0.64      0.54    401119
weighted avg       0.74      0.46      0.52    401119

---
Estimador: KNN
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     35000
           1       0.94      0.95      0.95     25000
           2       0.98      0.99      0.98     35000
           3       0.98      1.00      0.99    245000
           4       0.91      0.90      0.91  